In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
credits.shape

(4803, 4)

In [4]:
movies.shape

(4803, 20)

In [5]:
movies.merge(credits,on="title").shape

(4809, 23)

In [6]:
movies = movies.merge(credits,on="title")

In [7]:
movies = movies.drop(['budget','homepage','original_language','original_title','popularity',"production_companies","production_countries","release_date","revenue","runtime","spoken_languages",'status',"tagline","vote_average","vote_count","id"],axis=1)

In [8]:
movies.dropna(inplace=True)

In [9]:
import ast
def convert(obj):
  l = []
  for i in ast.literal_eval(obj):
    l.append(i["name"])
  return l

In [10]:
movies['genres']=movies['genres'].apply(convert)
movies['keywords'] =  movies['keywords'].apply(convert)

In [11]:
def convert3(obj):
  l = []
  counter = 0
  for i in ast.literal_eval(obj):
    if counter != 3:
      l.append(i["name"])
    else:
      break
  return l

In [12]:
movies["cast"]= movies["cast"].apply(convert3)

In [13]:
def fetch_director(obj):
  l = []
  for i in ast.literal_eval(obj):
    if i['job'] == "Director":
      l.append(i["name"])
      break
  return l

In [14]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [15]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [16]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "")for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "")for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "")for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "")for i in x])

In [17]:
movies["tags"] = movies["overview"] + movies["genres"]+ movies["cast"]+ movies["crew"]

In [18]:
new_df = movies[["movie_id","title","tags"]]

In [19]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x) )

C:\Users\jthor\AppData\Local\Temp\ipykernel_14184\449520859.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x) )


In [20]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\jthor\AppData\Local\Temp\ipykernel_14184\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [28]:
import nltk 
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [30]:
def stem(text):
    y = []
    for i in text.split():
      y.append(ps.stem(i))
    return " ".join(y)

In [31]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\jthor\AppData\Local\Temp\ipykernel_14184\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [32]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000,stop_words="english")
vectors = cv.fit_transform(new_df['tags']).toarray()

In [34]:
# cv.get_feature_names()

In [36]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [53]:
sorted(list(enumerate(similarity[1])),reverse=True,key = lambda x:x[1])[1:6]

[(12, 0.583383351196948),
 (199, 0.3481865296036271),
 (17, 0.20266980709997015),
 (973, 0.19446111706564928),
 (2521, 0.1917412472118426)]

In [74]:
new_df['title'] = new_df['title'].apply(lambda x:x.lower())
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse=True,key = lambda x:x[1])[1:6]
    # print(movie_index)
    # print(distances[movie_index])

    for i in movie_list:
        print(new_df.iloc[i[0]].title)

C:\Users\jthor\AppData\Local\Temp\ipykernel_14184\1177431912.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['title'] = new_df['title'].apply(lambda x:x.lower())


In [77]:
recommend("apollo 18".lower())


cj7
ed and his dead mother
the slaughter rule
modern times
tiny furniture


In [79]:
import pickle
pickle.dump(new_df,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))